In [ ]:
import glob
import json
import os
import shutil

import pandas
import utils
import yaml

import seabeepy as sb
from seabeepy.config import SETTINGS

In [ ]:
# Login to MinIO
client = sb.storage.minio_login(
    user=SETTINGS.MINIO_ACCESS_ID, password=SETTINGS.MINIO_SECRET_KEY
)

# Run ODM parameter combinations

Running ODM with different parameter options to explore orthophoto quality.

## 1. User input

Specify the mission to process and the version of the parameter combinations file to use.

In [ ]:
# Mission data to use
mission_dir = r"/home/notebook/shared-seabee-ns9879k/niva-tidy/2024/niva_202406260929_zosmap_olberg-s_rgb_30"

# Version of parameter combo file to use
param_version = 1

# Whether to delete existing results for this version of the parameters
delete_existing = True

## 2. Read parameter data

In [ ]:
# Read parameter file and count combinations
yaml_path = f"../data/odm_parameter_combos_v{param_version}.yaml"
n_combos = utils.count_param_combinations(yaml_path)
print(
    f"Version {param_version:02d} of the parameter file contains {n_combos} parameter combinations."
)
if n_combos < 1000:
    par_combos = utils.get_param_combinations(yaml_path)
else:
    raise ValueError(
        "Number of unique combinations is >999. Reduce the number of options in the parameter file."
    )

## 3. Copy data

Creates one copy of the mission data for each unique set of parameters.

In [ ]:
# Delete previous results if desired
res_dir = f"/home/notebook/shared-seabee-ns9879k/niva-tidy/2025/odm_tuning/params_v{param_version:02d}"
if os.path.isdir(res_dir) and delete_existing:
    shutil.rmtree(res_dir)

# Copy data for 'default' run
default_dir = utils.copy_mission_data(
    mission_dir, param_version, 0, client, odm_options=None
)

# Copy data for parameter combos
for run_id, odm_opts in enumerate(par_combos, start=1):
    combo_dir = utils.copy_mission_data(
        default_dir, param_version, run_id, client, odm_options=odm_opts
    )